newbie practice with shadowing

In [2]:
"""
Pickleball - Newbie (guided shadowing) + Experienced (visualize) modes

How to use:
- Run the script.
- Type 'newbie' or 'experienced' when prompted.
- In newbie mode you'll be asked which pose and handedness to train.
- Press 'q' to quit, 'r' to reset progress, 'h' to toggle handedness, 'n' to switch to next pose.
"""

import cv2
import mediapipe as mp
import numpy as np
import time

# -------------------------
# MediaPipe pose init
# -------------------------
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
POSE = mp_pose.Pose(static_image_mode=False,
                    model_complexity=1,
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5)

# -------------------------
# Utility helpers
# -------------------------
def lm_xy_px(landmarks, lm_enum, w, h, vis_thresh=0.25):
    """
    Convert a MediaPipe landmark enum to pixel (x, y).
    Returns None if the landmark is missing or has low visibility.
    """
    idx = lm_enum.value
    lm = landmarks[idx]
    if hasattr(lm, "visibility") and lm.visibility is not None and lm.visibility < vis_thresh:
        return None
    # landmark coordinates are normalized [0..1]
    return int(lm.x * w), int(lm.y * h)

def vec(a, b):
    """Return vector from a to b (b - a)."""
    return np.array([b[0]-a[0], b[1]-a[1]], dtype=np.float32)

def norm(v):
    """L2 norm with safe fallback."""
    v = np.array(v, dtype=np.float32)
    n = np.linalg.norm(v)
    if n < 1e-6:
        return v, 1e-6
    return v / n, n

# -------------------------
# Compute dynamic guide points
# -------------------------
def compute_guide_points(landmarks, frame_w, frame_h, pose_name="serve", handedness="right"):
    """
    Compute 3 guide (x,y) points in pixels for the selected pose based on the player's body.
    - landmarks: MediaPipe landmarks list
    - frame_w/frame_h: frame size in pixels
    - pose_name: 'serve', 'forehand', 'backhand'
    - handedness: 'right' or 'left'
    Returns: list of 3 (x,y) tuples. If landmarks insufficient, returns fallback centered points.
    """
    # get core landmarks (may return None)
    nose = lm_xy_px(landmarks, mp_pose.PoseLandmark.NOSE, frame_w, frame_h)
    ls = lm_xy_px(landmarks, mp_pose.PoseLandmark.LEFT_SHOULDER, frame_w, frame_h)
    rs = lm_xy_px(landmarks, mp_pose.PoseLandmark.RIGHT_SHOULDER, frame_w, frame_h)
    lh = lm_xy_px(landmarks, mp_pose.PoseLandmark.LEFT_HIP, frame_w, frame_h)
    rh = lm_xy_px(landmarks, mp_pose.PoseLandmark.RIGHT_HIP, frame_w, frame_h)
    lw = lm_xy_px(landmarks, mp_pose.PoseLandmark.LEFT_WRIST, frame_w, frame_h)
    rw = lm_xy_px(landmarks, mp_pose.PoseLandmark.RIGHT_WRIST, frame_w, frame_h)

    # if required landmarks are missing, provide safe fallback: center three vertical points
    if ls is None or rs is None or lh is None or rh is None:
        cx, cy = frame_w // 2, frame_h // 2
        return [(cx, int(cy*0.4)), (cx, cy), (cx, int(cy*1.4))]

    # compute body center/axes
    shoulder_center = ((ls[0] + rs[0]) / 2.0, (ls[1] + rs[1]) / 2.0)
    hip_center = ((lh[0] + rh[0]) / 2.0, (lh[1] + rh[1]) / 2.0)
    # horizontal axis (shoulder left -> right)
    horiz_vec_raw = vec(ls, rs)
    horiz_unit, shoulder_width = norm(horiz_vec_raw)
    # vertical axis (hip -> shoulder)
    up_vec_raw = vec(hip_center, shoulder_center)
    up_unit, body_height = norm(up_vec_raw)
    # dominant side
    side = 1 if handedness.lower().startswith("r") else -1

    # some scale constants tuned by heuristics
    w_factor = max(1.0, shoulder_width)  # shoulder width as pixels
    h_factor = max(1.0, body_height)

    # Serve: toss → contact → follow-through (roughly vertical toss + forward contact)
    if pose_name.lower().startswith("serve"):
        # toss: above head ~ shoulder_center + up * (body_height * 1.0)
        toss = (int(shoulder_center[0] + up_unit[0] * h_factor * 0.9),
                int(shoulder_center[1] + up_unit[1] * h_factor * 0.9))
        # contact: forward and slightly up from shoulder on dominant side
        contact = (int(shoulder_center[0] + horiz_unit[0] * (side * 0.7 * w_factor) + up_unit[0] * (-0.15 * h_factor)),
                   int(shoulder_center[1] + horiz_unit[1] * (side * 0.7 * w_factor) + up_unit[1] * (-0.15 * h_factor)))
        # follow: across body (opposite shoulder area)
        follow = (int(shoulder_center[0] + horiz_unit[0] * (side * -0.35 * w_factor) + up_unit[0] * (0.2 * h_factor)),
                  int(shoulder_center[1] + horiz_unit[1] * (side * -0.35 * w_factor) + up_unit[1] * (0.2 * h_factor)))
        return [toss, contact, follow]

    # Forehand: start (back), contact (front/shoulder), follow (across)
    if pose_name.lower().startswith("fore"):
        # start: slightly behind hip on dominant side
        start = (int(hip_center[0] + horiz_unit[0] * (side * 0.2 * w_factor) + up_unit[0] * (0.05 * h_factor)),
                 int(hip_center[1] + horiz_unit[1] * (side * 0.2 * w_factor) + up_unit[1] * (0.05 * h_factor)))
        # contact: in front of shoulder
        contact = (int(shoulder_center[0] + horiz_unit[0] * (side * 0.55 * w_factor)),
                   int(shoulder_center[1] + horiz_unit[1] * (side * 0.55 * w_factor)))
        # follow: across torso, slightly lower
        follow = (int(shoulder_center[0] + horiz_unit[0] * (side * -0.25 * w_factor) + up_unit[0] * (0.15 * h_factor)),
                  int(shoulder_center[1] + horiz_unit[1] * (side * -0.25 * w_factor) + up_unit[1] * (0.15 * h_factor)))
        return [start, contact, follow]

    # Backhand: mirror of forehand (dominant side considered above)
    if pose_name.lower().startswith("back"):
        # start: slightly behind hip on dominant side
        start = (int(hip_center[0] + horiz_unit[0] * (side * -0.2 * w_factor) + up_unit[0] * (0.05 * h_factor)),
                 int(hip_center[1] + horiz_unit[1] * (side * -0.2 * w_factor) + up_unit[1] * (0.05 * h_factor)))
        # contact: in front of shoulder (but for backhand across body)
        contact = (int(shoulder_center[0] + horiz_unit[0] * (side * -0.45 * w_factor)),
                   int(shoulder_center[1] + horiz_unit[1] * (side * -0.45 * w_factor)))
        # follow: across body
        follow = (int(shoulder_center[0] + horiz_unit[0] * (side * 0.3 * w_factor) + up_unit[0] * (0.12 * h_factor)),
                  int(shoulder_center[1] + horiz_unit[1] * (side * 0.3 * w_factor) + up_unit[1] * (0.12 * h_factor)))
        return [start, contact, follow]

    # default fallback if unknown pose
    cx, cy = frame_w // 2, frame_h // 2
    return [(cx, int(cy*0.4)), (cx, cy), (cx, int(cy*1.4))]

# -------------------------
# Newbie mode: guided shadowing
# -------------------------
def newbie_mode(camera_index=0):
    """
    Runs the interactive newbie trainer:
    - User selects pose and handedness.
    - The script computes three body-relative guide points and displays them.
    - The user moves their dominant wrist to hit each point in order.
    Controls:
    - q: quit
    - r: reset progress
    - n: next pose
    - h: toggle handedness
    """
    cap = cv2.VideoCapture(camera_index)
    if not cap.isOpened():
        print("ERROR: Camera not available.")
        return

    pose_choices = ["serve", "forehand", "backhand"]
    choice_idx = 0
    handedness = "right"  # default; user can toggle with 'h'
    progress = 0
    hit_cooldown = 0.4  # seconds freeze after hitting a point so user can prepare
    last_hit_time = 0.0
    radius = 18  # visual radius of guide point
    dist_thresh_px = 60  # how close wrist must be to count as hit (pixels) - adjust per camera

    print("NEWBIE MODE: press 'q' to quit, 'r' to reset, 'n' next pose, 'h' toggle hand")

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        h, w, _ = frame.shape

        # run mediapipe
        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = POSE.process(img_rgb)

        # compute guide points (dynamic) if we have landmarks
        guide_points = None
        if results.pose_landmarks:
            guide_points = compute_guide_points(results.pose_landmarks.landmark, w, h,
                                                pose_name=pose_choices[choice_idx],
                                                handedness=handedness)

        # draw guide points (use grey if not yet active, green if passed)
        for i, gp in enumerate(guide_points if guide_points is not None else []):
            color = (50, 50, 50)
            if i < progress:
                color = (0, 200, 0)
            elif i == progress:
                color = (0, 255, 255)
            cv2.circle(frame, gp, radius, color, -1)
            cv2.putText(frame, f"{i+1}", (gp[0]-8, gp[1]+6), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,0), 2)

        # draw wrist tracker and check proximity
        wrist = None
        if results.pose_landmarks:
            # choose dominant wrist
            if handedness == "right":
                wrist = lm_xy_px(results.pose_landmarks.landmark, mp_pose.PoseLandmark.RIGHT_WRIST, w, h)
            else:
                wrist = lm_xy_px(results.pose_landmarks.landmark, mp_pose.PoseLandmark.LEFT_WRIST, w, h)
            if wrist is not None:
                cv2.circle(frame, wrist, 10, (255, 255, 0), -1)

                # proximity detection (only if we have guide points and cooldown elapsed)
                if guide_points is not None and progress < len(guide_points) and (time.time() - last_hit_time) > hit_cooldown:
                    tx, ty = guide_points[progress]
                    d = np.linalg.norm(np.array(wrist, dtype=np.float32) - np.array([tx, ty], dtype=np.float32))
                    # scale threshold by shoulder width if possible (improve robustness)
                    # using a crude fallback if shoulder width not available
                    if d < dist_thresh_px:
                        progress += 1
                        last_hit_time = time.time()

        # Completion message
        if progress >= 3:
            cv2.putText(frame, f"{pose_choices[choice_idx].upper()} COMPLETE!", (30, 80),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.4, (0,255,0), 3)

        # UI text
        cv2.putText(frame, f"Mode: NEWBIE | Pose: {pose_choices[choice_idx]} | Hand: {handedness}",
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
        cv2.putText(frame, f"Progress: {progress}/3 (r reset, n next, h toggle hand, q quit)",
                    (10, h-20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (220,220,220), 1)

        # show frame
        cv2.imshow("Pickleball - Newbie Trainer", frame)
        key = cv2.waitKey(1) & 0xFF

        if key == ord("q"):
            break
        if key == ord("r"):
            progress = 0
            print("Progress reset.")
        if key == ord("n"):
            choice_idx = (choice_idx + 1) % len(pose_choices)
            progress = 0
            print("Switched to pose:", pose_choices[choice_idx])
        if key == ord("h"):
            handedness = "left" if handedness == "right" else "right"
            progress = 0
            print("Toggled handedness to", handedness)

    cap.release()
    cv2.destroyAllWindows()

# -------------------------
# Experienced mode (visualize / placeholder)
# -------------------------
def experienced_mode(camera_index=0):
    """
    Simple visualization mode for experienced players.
    Shows pose landmarks. Replace TODO with your LSTM/YOLO inference call to classify windows.
    """
    cap = cv2.VideoCapture(camera_index)
    if not cap.isOpened():
        print("ERROR: Camera not available.")
        return

    print("EXPERIENCED MODE: press 'q' to quit.")
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = POSE.process(img_rgb)

        if results.pose_landmarks:
            mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            # TODO: If you have a trained LSTM model, extract per-frame angles or keypoints here,
            # collect into sliding windows and call your LSTM to get predictions.
            # Example placeholder text:
            cv2.putText(frame, "Inference: (load model to enable classification)", (10,30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (200,200,0), 2)

        cv2.imshow("Pickleball - Experienced Visualizer", frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()

# -------------------------
# Entry
# -------------------------
if __name__ == "__main__":
    mode = input("Choose mode (newbie/experienced): ").strip().lower()
    if mode == "newbie":
        newbie_mode(camera_index=0)
    else:
        experienced_mode(camera_index=0)


EXPERIENCED MODE: press 'q' to quit.
